In [1]:
import wandb
import math
import random
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T
import pickle
import pandas as pd
import geopandas as gpd

import gnn_io

from my_gnn import GNN

# Abstract

This is the new version of the the notebook "wandb_two_channel_gnn". 

What we do in here:

1. Load wandb and set reasonable parameters.
2. Introduce key parameters: a. Car Volume in area where there was change, b. Car volume overall, c. Car volume per Edge.
3. Make predictions for these parameters. 

In [2]:
# Load the dictionary
with open('../results/result_dic.pkl', 'rb') as f:
    results_dict = pickle.load(f)

datasets = []
for key, df in results_dict.items():    
    if isinstance(df, pd.DataFrame):
        gdf = gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = "EPSG:2154"  # Assuming the original CRS is EPSG:2154
        gdf.to_crs("EPSG:4326", inplace=True)
        edge_index, car_volume_tensor, policy_tensor, nodes = gnn_io.create_edge_index_and_tensors(gdf)
        datasets.append((policy_tensor, car_volume_tensor))
    else:
        print(f"The value for key '{key}' is not a GeoDataFrame.")
        
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enatterer (tum-traffic-engineering). Use `wandb login --relogin` to force relogin


True

In [3]:
# max_value_total = 0
# mean_value_total = 0

# for policy_tensor, car_volume_tensor in datasets:
#     print(policy_tensor[:1])
#     print(car_volume_tensor[:1])
#     # Find the maximum and mean value of the car_volume_tensor
#     max_value = torch.max(car_volume_tensor)
#     mean_value = torch.mean(car_volume_tensor)
    
#     print(f"Max value of car_volume_tensor: {max_value.item()}")
#     print(f"Mean value of car_volume_tensor: {mean_value.item()}")
#     max_value_total = max(max_value_total, max_value)
#     mean_value_total = (mean_value_total + mean_value)/2
    
# print(max_value_total)
# print(mean_value_total)

In [4]:
def validate_model(model, valid_dl, loss_func):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0
    total = 0 
    with torch.inference_mode():
        total = 0  # Add a total counter for accuracy calculation
        for i, (policy_features, flow_targets) in enumerate(valid_dl):
             # Normalize data and labels
            policy_features = gnn_io.min_max_normalize(policy_features.float())  # Shape: [105, 31216, 3]
            flow_targets = gnn_io.min_max_normalize(flow_targets.float().unsqueeze(2)) # Shape: [105, 31216, 1]

            # Forward pass ➡
            outputs = model(edge_index, policy_features, flow_targets)
            # print("flow targets: ", flow_targets[:1])
            # print("outputs: ", outputs[:1])
            # Compute loss
            loss = loss_func(outputs, flow_targets)
            print(f"Loss: {loss.item()}")
            val_loss += loss.item() * flow_targets.size(0)  # Sum up the batch loss scaled by the number of examples
            print(f"Val loss: {val_loss}")
            total += flow_targets.size(0)  # Increment total by the number of labels
            
    average_val_loss = val_loss/total
    return average_val_loss  # Average loss over all examples, accuracy as correct predictions over total

In [5]:
policy_input_dim = 3  # Dimensionality of policy features: capacity, freeflow speed, modes
traffic_input_dim = 1  # Dimensionality of traffic flow features
hidden_dim = 32  # Dimensionality of hidden representations

device = "cuda:0" if torch.cuda.is_available() else "cpu"

gnn = GNN(policy_input_dim=3, traffic_input_dim=1, hidden_dim=8)
gnn.apply(gnn_io.init_weights)

MyGNN()

In [6]:
# Launch 5 experiments, trying different dropout rates

learning_rates = [1e-5]  # List of learning rates to iterate over
# ,  1e-4, 1e-3, 1e-2, 1e-1
for lr in learning_rates:
    # 🐝 initialise a wandb run
    wandb.init(
        project="gnn_version_4",
        config={
            "epochs": 20,
            "batch_size": 20,
            "lr": lr
    })
    
    # Copy your config 
    config = wandb.config

    # Get the data
    train_dl = gnn_io.get_dataloader(is_train=True, batch_size=config.batch_size, datasets=datasets)
    valid_dl = gnn_io.get_dataloader(is_train=False, batch_size=config.batch_size, datasets=datasets)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)
    
    # Make the loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(gnn.parameters(), lr=config.lr)

   # Training
    example_ct = 0
    step_ct = 0
    for epoch in range(config.epochs):
        gnn.train()
        for step, (policy_features, car_volume) in enumerate(train_dl):            
            policy_features_normalized, car_volume_normalized = gnn_io.min_max_normalize(policy_features.float()), gnn_io.min_max_normalize(car_volume.float().unsqueeze(2))        
            predicted_car_volume = gnn(edge_index, policy_features_normalized)
            # print("Policy features: ", policy_features_normalized[:1])
            # print("Normalized car volume: ", car_volume_normalized[:1])
            # print("Predicted car volume: ", predicted_car_volume[:1])
            train_loss = loss_func(predicted_car_volume, car_volume_normalized)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            example_ct += len(policy_features_normalized)
            print("Predicted car volume ")
            print(predicted_car_volume[:1])
            print(predicted_car_volume)
            print(predicted_car_volume.shape)
            break
        break
    #         metrics = {"train/train_loss": train_loss, 
    #                    "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch, 
    #                    "train/example_ct": example_ct}
    #         if step + 1 < n_steps_per_epoch:
    #             # 🐝 Log train metrics to wandb 
    #             wandb.log(metrics)
    #         step_ct += 1

    #     val_loss = validate_model(gnn, valid_dl, loss_func)
    #     val_metrics = {"val/val_loss": val_loss}
    #     wandb.log({**metrics, **val_metrics})
    # wandb.finish()

Predicted car volume 
tensor([[[-0.0005],
         [-0.0454],
         [-0.1421],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]]], grad_fn=<SliceBackward0>)
tensor([[[-0.0005],
         [-0.0454],
         [-0.1421],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[-0.0005],
         [-0.0848],
         [-0.1737],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[-0.0003],
         [-0.0293],
         [-0.1259],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        ...,

        [[-0.0002],
         [-0.1243],
         [-0.1263],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[-0.0003],
         [-0.0214],
         [-0.0996],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[-0.0003],
         [-0.0214],
         [-0.0996],
         ...,
         [ 0.0000],
         [ 0.0000],
        

In [7]:
predicted_car_volume

tensor([[[-0.0005],
         [-0.0454],
         [-0.1421],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[-0.0005],
         [-0.0848],
         [-0.1737],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[-0.0003],
         [-0.0293],
         [-0.1259],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        ...,

        [[-0.0002],
         [-0.1243],
         [-0.1263],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[-0.0003],
         [-0.0214],
         [-0.0996],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]],

        [[-0.0003],
         [-0.0214],
         [-0.0996],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]]], grad_fn=<ViewBackward0>)

In [8]:
# iterate over the twenty entries (the first dimension of the tensor)
def get_overall_car_volume(car_volume_normalized, predicted_car_volume, batch_size):
    absolute_diff = 0
    for i in range(batch_size):
        print(" ")
        actual_sum = sum(car_volume_normalized[i])
        print(actual_sum)
        predicted_sum = sum(predicted_car_volume[i])
        print(predicted_sum)
        diff = abs(actual_sum - predicted_sum)
        print(diff)
        absolute_diff += diff
    print(absolute_diff)

In [9]:
get_overall_car_volume(car_volume_normalized, predicted_car_volume, 20)

 
tensor([995.3181])
tensor([-1766.6842], grad_fn=<AddBackward0>)
tensor([2762.0022], grad_fn=<AbsBackward0>)
 
tensor([821.2917])
tensor([-1467.3787], grad_fn=<AddBackward0>)
tensor([2288.6704], grad_fn=<AbsBackward0>)
 
tensor([767.1666])
tensor([-1366.2631], grad_fn=<AddBackward0>)
tensor([2133.4297], grad_fn=<AbsBackward0>)
 
tensor([835.9867])
tensor([-1495.8457], grad_fn=<AddBackward0>)
tensor([2331.8325], grad_fn=<AbsBackward0>)
 
tensor([853.7370])
tensor([-1529.9232], grad_fn=<AddBackward0>)
tensor([2383.6602], grad_fn=<AbsBackward0>)
 
tensor([967.0150])
tensor([-1716.1787], grad_fn=<AddBackward0>)
tensor([2683.1938], grad_fn=<AbsBackward0>)
 
tensor([894.3200])
tensor([-1582.3336], grad_fn=<AddBackward0>)
tensor([2476.6536], grad_fn=<AbsBackward0>)
 
tensor([498.1046])
tensor([-918.9207], grad_fn=<AddBackward0>)
tensor([1417.0254], grad_fn=<AbsBackward0>)
 
tensor([674.7240])
tensor([-1208.0469], grad_fn=<AddBackward0>)
tensor([1882.7709], grad_fn=<AbsBackward0>)
 
tensor([5